In [5]:
import pymongo
import pandas as pd
import random
import json
from os.path import join,dirname
import os
from pymongo import MongoClient
import random
from datetime import date, timedelta, datetime
DATEFORMAT = "%Y-%m-%d"

client = MongoClient('localhost', 27017)
db = client['aion']
collection = db['employee']

In [6]:
ROOT_DIR = os.path.abspath(os.curdir)
filename =join(ROOT_DIR, 'data/employee_mock_data.json')
if filename:
    with open(filename, 'r') as f:
        employees = json.load(f)
print(employees[0:2])
# bias salary by age and by gender
employee_list = []
employee_pay = []
employee_gender_lst = []
employee_age_lst = []
for key,employee in enumerate(employees):
    employee_list.append(employee['name'])
    employee_gender_lst.append(employee['gender'].lower())
    if employee['hourly_rate'] < 0:
        employee['hourly_rate'] = 20
    if employee['gender'].lower() == 'female':
        employee['hourly_rate'] *= random.choice([1.3, 1.6, 1.4, 1.1])
    dob = datetime.strptime(employee['dob'],DATEFORMAT)
    employee['dob'] = datetime(dob.year,dob.month,dob.day,0,0,0)
    age = int(round((datetime.today() - dob).days / 365,0))
    employee_age_lst.append(age)
    employee['hourly_rate'] += round((datetime.today() - employee['dob']).days/900,2)
    employee_pay.append(round(employee['hourly_rate'],2))
print(employees[0:2])
employee_pay_dct = dict(zip(employee_list,employee_pay))
employee_gender_dct = dict(zip(employee_list,employee_gender_lst))
employee_age_dct = dict(zip(employee_list,employee_age_lst))
print(employee_pay)
#collection.insert_many(employees)


[{'id': 1, 'name': 'Cindra Serotsky', 'department': 'Marketing', 'title': 'Senior Cost Accountant', 'gender': 'Female', 'dob': '1993-06-09', 'hourly_rate': 14}, {'id': 2, 'name': 'Nataline Marples', 'department': 'Business Development', 'title': 'Actuary', 'gender': 'Female', 'dob': '1996-09-07', 'hourly_rate': 20}]
[{'id': 1, 'name': 'Cindra Serotsky', 'department': 'Marketing', 'title': 'Senior Cost Accountant', 'gender': 'Female', 'dob': datetime.datetime(1993, 6, 9, 0, 0), 'hourly_rate': 25.910000000000004}, {'id': 2, 'name': 'Nataline Marples', 'department': 'Business Development', 'title': 'Actuary', 'gender': 'Female', 'dob': datetime.datetime(1996, 9, 7, 0, 0), 'hourly_rate': 35.19}]
[25.91, 35.19, 26.68, 45.56, 37.56, 44.62, 44.86, 31.04, 30.53, 39.94, 52.25, 37.28, 30.42, 38.61, 29.7, 43.12, 41.37, 41.21, 32.49, 38.99, 49.49, 38.94, 56.45, 32.33, 44.79, 47.46, 34.17, 36.9, 46.31, 27.42, 33.75, 29.72, 24.73, 18.78, 60.01, 56.87, 39.06, 36.63, 34.18, 48.52, 26.98, 29.08, 31.21,

In [ ]:
import numpy as np
from scipy.stats import gamma

# fill projects
collection = db['project_type']
dct_type = {
    'type':['research','innovation','audit','reconciliation','conference','manufacturing',
            'construction']
}
#df_type = pd.DataFrame.from_dict(dct_type)
#data_json = json.loads(df_type.to_json(orient='records'))
#collection.insert_many(data_json)

types = dct_type['type']

collection_workers = db['project_task']
tasks_list = []
project_lists = []
composite_lst = []
for idx in range(1,5000):
    dct = {}
    composite={}
    # determine project manager
    manager = random.choice(employee_list)
    manager_gender = employee_gender_dct[manager].lower()
    manager_age = employee_age_dct[manager]
    dct['manager'] = manager
    dct['manager_gender'] = manager_gender
    dct['manager_age'] = manager_age
    # generate project type
    name = 'Project '+str(idx)
    dct['name'] = name
    dct['type'] = random.choice(types)
    s = random.choice(['closed','closed','open'])
    # generate dates
    first_date = datetime(2013,1,1,0,0,0)
    days = (datetime.today() - first_date).days
    sp = first_date + timedelta(days=random.choice(list(range(0,days))))
    
    dct['startdate_proposed'] = sp
    proj_len = int(gamma.rvs(a=60, size=1))
    if proj_len < 0:
        proj_len *= -1
    ep = sp + timedelta(days=proj_len)
    dct['enddate_proposed'] = ep
    
    s = 'closed'
    if ep > datetime.today():
        s = 'open'
    dct['status'] = s

    delay = int(gamma.rvs(a=3, size=1))
    if manager_gender == 'female':
            delay *= .7
    delay = delay * (65 - manager_age)*.02 # older is better
    if delay < 0:
        delay = 0
    sa = sp + timedelta(days=delay)
    dct['startdate_actual'] = sa
    # enddate delay
    if s == 'closed':
        delay = int(gamma.rvs(a=10, size=1))
        delay *= random.choice([-1,1,1])
        if manager_gender == 'female':
            delay *= .4
        if delay < 0:
            delay = delay * manager_age*1.4 # older is better
        else:
            delay = delay * (65-manager_age)*.03 # older is better
        
        if delay < 0:
            delay *= -1
        dct['enddate_actual'] = ep + timedelta(days=delay)

    else:
        dct['enddate_actual'] = datetime(2020,1,1,0,0,0)
    
    project_lists.append(dct)
    
    ####################### tasks
    emps = int(gamma.rvs(a=20, size=1))
    for emp in range(0,emps):
        composite = {}
        dct_tasks = {}
        dct_tasks['project'] =  name
        task_len = proj_len * 8 *.5# hours
        dct_tasks['employee'] = random.choice(employee_list)
        
        start = sa + timedelta(days=random.choice(list(range(0,int(proj_len*.8)))))
        dct_tasks['start'] = start
        end = start + timedelta(hours= int(gamma.rvs(a=task_len, size=1)))
        if end > dct['enddate_actual']:
            end = dct['enddate_actual']
        dct_tasks['end'] = end
        duration = (dct_tasks['end'] - dct_tasks['start']).days*8
        composite = {
            'project':name,
            'manager':manager,
            'manager_gender':manager_gender,
            'manager_age':manager_age,
            'type':dct['type'],
            'status':dct['status'],
            'startdate_proposed':dct['startdate_proposed'],
            'startdate_actual':dct['startdate_actual'],
            'enddate_proposed':dct['enddate_proposed'],
            'enddate_actual':dct['enddate_actual'],
            'employee':dct_tasks['employee'],
            'gender': employee_gender_dct[dct_tasks['employee']].lower(),
            'start':dct_tasks['start'],
            'end':dct_tasks['end'],
            'task_duration':round(duration,2),
            'remuneration':round(employee_pay_dct[dct_tasks['employee']]*abs(duration),2),
            # find the delay
            'delay_start': (dct['startdate_actual'] - dct['startdate_proposed']).days*8,
            'delay_end':(dct['enddate_actual'] - dct['enddate_proposed']).days*8,
            'project_duration': (dct['enddate_actual'] - dct['startdate_actual']).days
            
        }
        tasks_list.append(dct_tasks)
        composite_lst.append(composite)

collection_project = db['project']
collection_project.insert_many(project_lists)
collection_workers.insert_many(tasks_list)
collection_composite = db['project_composite']
collection_composite.insert_many(composite_lst)


KeyError: 'enddate_actual'